# PyTorch: Learn the Basics

* Tensors
* Datasets & DataLoaders
* Transforms
* Build Model
* Autograd
* Optimization
* Save & Load Model

# Tensors

Why this instead of numpy ndarrays? Tensors can run on GPUs. They can also be optimized for automatic differentiation.


In [121]:
import torch
import numpy as np

In [122]:
%load_ext watermark

## Initializing a tensor

In [2]:
# directly from data
data = [[1, 2],[3, 4]]
x_data = torch.tensor(data)

In [3]:
type(x_data)

torch.Tensor

In [7]:
# from a numpy array
np_array = np.array(data)
x_np = torch.from_numpy(np_array)

In [9]:
# creating new tensors from another tensor (e.g. maintains shape)
x_ones = torch.ones_like(x_data) # retains the properties of x_data
print(f"Ones Tensor: \n {x_ones} \n")


Ones Tensor: 
 tensor([[1, 1],
        [1, 1]]) 



In [22]:
# a random tensor, values between 0 and 1
x_rand = torch.rand_like(x_data, dtype=torch.float) # overrides the datatype of x_data
print(f"Random Tensor: \n {x_rand} \n")

Random Tensor: 
 tensor([[0.2791, 0.5894],
        [0.2706, 0.9686]]) 



In [25]:
# constant or random values
shape = (2,3,)
zeros_tensor = torch.zeros(shape)
ones_tensor = torch.ones(shape)
rand_tensor = torch.rand(shape)

print(f"Zeros Tensor: \n {zeros_tensor}")
print(f"Ones Tensor: \n {ones_tensor} \n")
print(f"Random Tensor: \n {rand_tensor} \n")


Zeros Tensor: 
 tensor([[0., 0., 0.],
        [0., 0., 0.]])
Ones Tensor: 
 tensor([[1., 1., 1.],
        [1., 1., 1.]]) 

Random Tensor: 
 tensor([[0.8237, 0.4671, 0.9021],
        [0.8525, 0.0439, 0.2446]]) 



## Attributes

In [32]:
tensor = torch.rand(3,4)
print(tensor)
print(f"Shape of tensor: {tensor.shape}")
print(f"Datatype of tensor: {tensor.dtype}")
print(f"Device tensor is stored on: {tensor.device}")

tensor([[0.2505, 0.4804, 0.4175, 0.0060],
        [0.2331, 0.1359, 0.9607, 0.8692],
        [0.5707, 0.8791, 0.7290, 0.4179]])
Shape of tensor: torch.Size([3, 4])
Datatype of tensor: torch.float32
Device tensor is stored on: cpu


In [52]:
# show some random attributes
from random import sample
sample([attribute for attribute in dir(tensor) if '_' not in attribute], 20)

['signbit',
 'adjoint',
 'aminmax',
 'atanh',
 'erfinv',
 'kthvalue',
 'cdouble',
 'hardshrink',
 'divide',
 'pinverse',
 'item',
 'xlogy',
 'arccos',
 'svd',
 'expand',
 'flipud',
 'movedim',
 'chalf',
 'long',
 'equal']

In [53]:
tensor.bernoulli()

tensor([[0., 1., 0., 0.],
        [1., 0., 1., 1.],
        [0., 1., 1., 1.]])

## Operations on Tensors

While the tensors can be run on GPU, by default, they're created on the CPU. Operations can be run on CPU and an [Accelerator](https://docs.pytorch.org/docs/stable/torch.html#accelerators), a `torch.device` like [CUDA](https://docs.pytorch.org/docs/stable/cuda.html) or [MPS](https://docs.pytorch.org/docs/stable/mps.html).

NOTE: To use GPU acceleration in a dev container:
- Remote Server with NVIDIA GPU (CUDA): The most common and effective way to get GPU acceleration in a Dev Container is if your Dev Container is connected to a remote server that has an NVIDIA GPU. In this scenario, you would configure the Dev Container to expose that GPU to the container, and your code would use torch.device("cuda"). This is a very popular setup for professional ML development.
- Run locally, outside the container, to use MPS.

In [62]:
# Check if CUDA (NVIDIA GPU) is available
if torch.cuda.is_available():
    device = torch.device("cuda")
    print(f"Using GPU: {torch.cuda.get_device_name(0)}")
# elif torch.mps.is_available():  # note this fails in a devcontainer run on mac because it's linux
elif torch.backends.mps.is_available():
    device = torch.device("mps")
    print(f"Using MPS: {torch.mps.get_device_name(0)}")
else:
    device = torch.device("cpu")
    print("No GPU found, using CPU.")

No GPU found, using CPU.


In [54]:
# We move our tensor to the current accelerator if available
if torch.accelerator.is_available():
    tensor = tensor.to(torch.accelerator.current_accelerator())

In [65]:
tensor

tensor([[0.2505, 0.4804, 0.4175, 0.0060],
        [0.2331, 0.1359, 0.9607, 0.8692],
        [0.5707, 0.8791, 0.7290, 0.4179]])

In [68]:
# standard indexing

print(tensor[1,0])
print(tensor[:, 2])

tensor(0.2331)
tensor([0.4175, 0.9607, 0.7290])


In [70]:
# joining tensors
t1 = torch.cat([tensor, tensor, tensor], dim=1)
print(t1)

tensor([[0.2505, 0.4804, 0.4175, 0.0060, 0.2505, 0.4804, 0.4175, 0.0060, 0.2505,
         0.4804, 0.4175, 0.0060],
        [0.2331, 0.1359, 0.9607, 0.8692, 0.2331, 0.1359, 0.9607, 0.8692, 0.2331,
         0.1359, 0.9607, 0.8692],
        [0.5707, 0.8791, 0.7290, 0.4179, 0.5707, 0.8791, 0.7290, 0.4179, 0.5707,
         0.8791, 0.7290, 0.4179]])


Arithmetic operations

In [ ]:
# matrix multiplication, 3 options
y1 = tensor @ tensor.T
y1

tensor([[0.4679, 0.5300, 0.8722],
        [0.5300, 1.7512, 1.3162],
        [0.8722, 1.3162, 1.8048]])

In [74]:
y2 = tensor.matmul(tensor.T)
y2

tensor([[0.4679, 0.5300, 0.8722],
        [0.5300, 1.7512, 1.3162],
        [0.8722, 1.3162, 1.8048]])

In [ ]:
# randomize in the shape of y1
y3 = torch.rand_like(y1)
y3

tensor([[0.4437, 0.4634, 0.6674],
        [0.0069, 0.6388, 0.2411],
        [0.8581, 0.1487, 0.6242]])

In [ ]:
# note the `out`
torch.matmul(tensor, tensor.T, out=y3)

tensor([[0.4679, 0.5300, 0.8722],
        [0.5300, 1.7512, 1.3162],
        [0.8722, 1.3162, 1.8048]])

In [ ]:
# y3 gets overwritten
y3

tensor([[0.4679, 0.5300, 0.8722],
        [0.5300, 1.7512, 1.3162],
        [0.8722, 1.3162, 1.8048]])

In [95]:
# y1 == y2  # will fail

# test for content and shape
torch.equal(y1, y2)

True

In [96]:
# == element wise comparison
torch.all(y1 == y2) # analagous to np.all

tensor(True)

In [98]:
# test if all 3 are equal
assert torch.equal(y1, y2) and torch.equal(y2, y3)

In [99]:
# Element-wise product, also 3 options
z1 = tensor * tensor
z2 = tensor.mul(tensor)

z3 = torch.rand_like(tensor)
torch.mul(tensor, tensor, out=z3)

tensor([[6.2736e-02, 2.3083e-01, 1.7430e-01, 3.6021e-05],
        [5.4333e-02, 1.8469e-02, 9.2296e-01, 7.5548e-01],
        [3.2569e-01, 7.7290e-01, 5.3151e-01, 1.7467e-01]])

In [100]:
assert torch.equal(z1, z2) and torch.equal(z2, z3)

In [ ]:
# convert a single element tensor into a python numerical value
tensor.sum()

tensor(5.9501)

In [104]:
print(tensor.sum().item(), type(tensor.sum().item()))

5.950121879577637 <class 'float'>


## Bridge with NumPy

Move to/from numpy array with `.numpy()` and `torch.from_numpy(x)`

In [117]:
# Convert tensor to numpy array
tensor.numpy()

array([[5.2504706, 5.4804497, 5.417492 , 5.006002 ],
       [5.2330947, 5.1359005, 5.960708 , 5.8691816],
       [5.5706916, 5.879149 , 5.7290444, 5.4179378]], dtype=float32)

In [115]:
# Convert numpy array to tensor
torch.from_numpy(np.random.random((3,4)))

tensor([[0.4286, 0.0684, 0.5026, 0.9145],
        [0.8174, 0.0451, 0.3830, 0.1587],
        [0.1986, 0.6687, 0.3983, 0.3684]], dtype=torch.float64)

# Datasets and Dataloaders

In [123]:
%watermark

Last updated: 2025-08-06T23:14:48.207516+00:00

Python implementation: CPython
Python version       : 3.12.11
IPython version      : 9.4.0

Compiler    : GCC 12.2.0
OS          : Linux
Release     : 6.10.14-linuxkit
Machine     : aarch64
Processor   : 
CPU cores   : 7
Architecture: 64bit



In [124]:
%watermark -iv

numpy    : 2.3.1
torch    : 2.7.1
watermark: 2.5.0

